# Sentence Splitting Experiments
Here we examine the quality of the sentence splitting in the corpus.
We found that the sentence splitting in the corpus which was done with stanfordnlp was too aggressive. Many sentences were split incorrectly and often trigger-argument pairs (from the document level annotation) ended up being in different sentences.
We compare negative labeling functions only using the sentence splitting information from stanfordnlp vs. SoMaJo and we manually examine the sentence splitting quality on the incorrectly labeled instances.

In [ ]:
import sys
sys.path.append("../")

## spaCy, stanfordnlp, SoMaJo comparison

Download German spaCy core model
```shell script
python -m spacy download de_core_news_md
```

In [2]:
import spacy
nlp_spacy = spacy.load('de_core_news_md')

In [3]:
import stanfordnlp
# stanfordnlp.download('de') # uncomment if this is the first run
nlp_stanford = stanfordnlp.Pipeline(lang='de')

Use device: cpu
---
Loading: tokenize
With settings: 
{'model_path': '/Users/phuc/stanfordnlp_resources/de_gsd_models/de_gsd_tokenizer.pt', 'lang': 'de', 'shorthand': 'de_gsd', 'mode': 'predict'}
---
Loading: mwt
With settings: 
{'model_path': '/Users/phuc/stanfordnlp_resources/de_gsd_models/de_gsd_mwt_expander.pt', 'lang': 'de', 'shorthand': 'de_gsd', 'mode': 'predict'}
Building an attentional Seq2Seq model...
Using a Bi-LSTM encoder
Using soft attention for LSTM.
Finetune all embeddings.
---
Loading: pos
With settings: 
{'model_path': '/Users/phuc/stanfordnlp_resources/de_gsd_models/de_gsd_tagger.pt', 'pretrain_path': '/Users/phuc/stanfordnlp_resources/de_gsd_models/de_gsd.pretrain.pt', 'lang': 'de', 'shorthand': 'de_gsd', 'mode': 'predict'}
---
Loading: lemma
With settings: 
{'model_path': '/Users/phuc/stanfordnlp_resources/de_gsd_models/de_gsd_lemmatizer.pt', 'lang': 'de', 'shorthand': 'de_gsd', 'mode': 'predict'}
Building an attentional Seq2Seq model...
Using a Bi-LSTM encoder
Usi

In [4]:
from somajo import SoMaJo
tokenizer = SoMaJo("de_CMC", split_camel_case=True)

### Test documents

In [6]:
doc1 = "#S1 Nach der Weichenstörung in Hohen Neuendorf verkehren die S-Bahnen wieder durchgehend, erster Zug ab #Frohnau 21:58 Uhr und erster Zug ab #Hohen_Neuendorf 22:03 Uhr."
doc2 = "Unfall\nAbschnitt: Marzahn (Berlin)\nGültig ab: 09.02.2016 20:06\ngesperrt, Unfall\n"
doc3 = "■ #A1 #Bremen Richtung #Hamburg zwischen Horster Dreieck und #Stillhorn 9 km #Stau.  Dort ist wegen #Bauarbeiten nur eine Spur frei.\n"
doc4 = "Wegen einer techn. Störung an der Strecke besteht für die Linien S41, S42 u. S46 zw. Halensee <> Westkreuz <> Messe Nord <> Westend S-Bahn-Pendelverkehr im 20-Minuten-Takt. Die Linien S41 u. S42 fahren nur im 10-Minuten-Takt, die Linie S46 fährt nur Königs Wusterhausen <> Tempelhof."
doc5 = "#S3, #S5, #S7, #S9: Nach einer ärztliche Versorgung eines Fahrgastes im Zug in Bellevue kommt es noch zu Verspätungen und vereinzelten Ausfällen."

In [7]:
test_docs = [doc1, doc2, doc3, doc4, doc5]

### Process documents with spaCy, stanfordnlp, somajo

In [8]:
spacy_docs = [nlp_spacy(doc) for doc in test_docs]

In [9]:
stanford_docs = [nlp_stanford(doc) for doc in test_docs]

/tmp/pip-req-build-u9mddw1z/aten/src/ATen/native/LegacyDefinitions.cpp:19: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead.
/tmp/pip-req-build-u9mddw1z/aten/src/ATen/native/LegacyDefinitions.cpp:19: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead.
/tmp/pip-req-build-u9mddw1z/aten/src/ATen/native/LegacyDefinitions.cpp:19: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead.
/tmp/pip-req-build-u9mddw1z/aten/src/ATen/native/LegacyDefinitions.cpp:19: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead.
/tmp/pip-req-build-u9mddw1z/aten/src/ATen/native/LegacyDefinitions.cpp:19: UserWarning: masked_fill_ received a 

In [11]:
somajo_docs = [list(tokenizer.tokenize_text([doc])) for doc in test_docs]

### Tokenization comparison
How to access tokens:

#### spaCy
`Doc` is a sequence of `Token`s. We can get the token text with `Token.text`.

#### stanfordnlp
Here we have to access the sentences of a `Doc` to access the tokens with `tokens` property. We can get the token text with `Token.text`.

#### somajo
Similar to stanfordnlp.

In [12]:
def get_spacy_doc_tokens(doc):
    return [token.text for token in doc]

def get_stanford_doc_tokens(doc):
    return [token.text for sentence in doc.sentences for token in sentence.tokens]

def get_somajo_doc_tokens(doc):
    return [token.text for sentence in doc for token in sentence]

In [13]:
for spacy_doc, stanford_doc, somajo_doc in zip(spacy_docs, stanford_docs, somajo_docs):
    spacy_tokens = get_spacy_doc_tokens(spacy_doc)
    print("spaCy:", spacy_tokens)
    stanford_tokens = get_stanford_doc_tokens(stanford_doc)
    print("stanfordnlp:", stanford_tokens)
    somajo_tokens = get_somajo_doc_tokens(somajo_doc)
    print("somajo:", somajo_tokens)
    print("\n")

spaCy: ['#', 'S1', 'Nach', 'der', 'Weichenstörung', 'in', 'Hohen', 'Neuendorf', 'verkehren', 'die', 'S-Bahnen', 'wieder', 'durchgehend', ',', 'erster', 'Zug', 'ab', '#', 'Frohnau', '21:58', 'Uhr', 'und', 'erster', 'Zug', 'ab', '#', 'Hohen_Neuendorf', '22:03', 'Uhr', '.']
stanfordnlp: ['#S1', 'Nach', 'der', 'Weichenstörung', 'in', 'Hohen', 'Neuendorf', 'verkehren', 'die', 'S-', 'Bahnen', 'wieder', 'durchgehend', ',', 'erster', 'Zug', 'ab', '#', 'Frohnau', '21:58', 'Uhr', 'und', 'erster', 'Zug', 'ab', '#', 'Hohen_Neuendorf', '22:03', 'Uhr', '.']
stanza: ['#S1', 'Nach', 'der', 'Weichenstörung', 'in', 'Hohen', 'Neuendorf', 'verkehren', 'die', 'S-Bahnen', 'wieder', 'durchgehend', ',', 'erster', 'Zug', 'ab', '#', 'Frohnau', '21:58', 'Uhr', 'und', 'erster', 'Zug', 'ab', '#', 'Hohen_Neuendorf', '22:03', 'Uhr', '.']
somajo: ['#S1', 'Nach', 'der', 'Weichenstörung', 'in', 'Hohen', 'Neuendorf', 'verkehren', 'die', 'S-Bahnen', 'wieder', 'durchgehend', ',', 'erster', 'Zug', 'ab', '#Frohnau', '21:58'

spaCy tokenizer treats hashtags as separate tokens and keeps whitespace characters.
stanfordnlp more often than not treats hashtags as separate token and often does not handle abbreviations well, i.e. the tokenizer treats the dot as a separate token.
It also tends to split words containing punctuation marks more aggressively than the other tokenizers.
SoMaJo does not treat hashtags as separate tokens and handles abbreviations better. It does however split dates into multiple tokens.

### Sentence splitting comparison

In [14]:
def get_spacy_doc_sentences(doc):
    return [s.text for s in doc.sents]

def get_stanford_doc_sentences(doc):
    # introduces whitespaces
    # see: https://github.com/stanfordnlp/stanfordnlp/blob/dev/stanfordnlp/models/common/doc.py
    # to get original sentence text
    return [" ".join([token.text for token in sentence.tokens]) for sentence in doc.sentences]

def get_somajo_doc_sentences(doc):
    # introduces whitespaces
    return [" ".join([token.text for token in sentence]) for sentence in doc]

In [15]:
for spacy_doc, stanford_doc, somajo_doc in zip(spacy_docs, stanford_docs, somajo_docs):
    spacy_sentences = get_spacy_doc_sentences(spacy_doc)
    print("spaCy:", len(spacy_sentences), "\n", spacy_sentences)
    stanford_sentences = get_stanford_doc_sentences(stanford_doc)
    print("stanfordnlp:", len(stanford_sentences), "\n", stanford_sentences)
    somajo_sentences = get_somajo_doc_sentences(somajo_doc)
    print("somajo:", len(somajo_sentences), "\n", somajo_sentences)
    print("\n")

spaCy: 7 
 ['#S1', 'Nach der Weichenstörung in Hohen Neuendorf verkehren die S-Bahnen wieder durchgehend, erster Zug ab', '#', 'Frohnau', '21:58 Uhr und erster Zug ab', '#Hohen_Neuendorf', '22:03 Uhr.']
stanfordnlp: 1 
 ['#S1 Nach der Weichenstörung in Hohen Neuendorf verkehren die S- Bahnen wieder durchgehend , erster Zug ab # Frohnau 21:58 Uhr und erster Zug ab # Hohen_Neuendorf 22:03 Uhr .']
stanza: 1 
 ['#S1 Nach der Weichenstörung in Hohen Neuendorf verkehren die S-Bahnen wieder durchgehend, erster Zug ab #Frohnau 21:58 Uhr und erster Zug ab #Hohen_Neuendorf 22:03 Uhr.']
somajo: 1 
 ['#S1 Nach der Weichenstörung in Hohen Neuendorf verkehren die S-Bahnen wieder durchgehend , erster Zug ab #Frohnau 21:58 Uhr und erster Zug ab #Hohen_Neuendorf 22:03 Uhr .']


spaCy: 5 
 ['Unfall\nAbschnitt: Marzahn (Berlin)\n', 'Gültig ab', ':', '09.02.2016', '20:06\ngesperrt, Unfall\n']
stanfordnlp: 1 
 ['Unfall Abschnitt : Marzahn ( Berlin ) Gültig ab : 09.02.2016 20:06 gesperrt , Unfall']
stanza: 

In the small sample of sentences we can observe that spaCy tends to split the document text very aggressively. It seems to not be able to handle hashtags, punctuation marks and abbreviations well.
stanfordnlp tends to do a little better, but seems rather ill-equipped to handle text data from social media containing a lot of abbreviations and use of special punctuation marks.
SoMaJo does considerably better. In our testing we found that it only made mistakes on very few occasions where it encountered unknown abbreviations.
Therefore we chose to do event extraction on a document level and use SoMaJo sentence splitting information for our negative labeling functions.

## Automatic approach to evaluate sentence splitting quality
In order to automatically evaluate the quality of sentence splitting of stanfordnlp and SoMaJo we compare event role labeling functions, that label an example as `no_arg` when trigger and (potential) argument are in separate sentences according to the sentence splitting (boundary) information and abstain otherwise.

Caveats: If the annotators did not pay attention to sentence boundaries when labeling event roles, then it may seem that the splitter made a mistake. This would then measure the consistency / quality of the annotation rather than the quality of the sentence splitter. This approach only covers the sentence splitting errors where trigger and argument ended up in different sentences according to the splitter. There may be other sentence splitting errors where trigger and argument still ended up in the same sentence or splitting errors in sentences with no event roles.

In [18]:
import pandas as pd
from wsee.utils import corpus_statistics
sd4m_train = pd.read_json("../data/daystream_corpus/train/train_with_events_and_defaults.jsonl", lines=True, encoding='utf8')
filtered_sd4m_train = sd4m_train[sd4m_train.apply(lambda document: corpus_statistics.has_triggers(document), axis=1)]
corpus_statistics.get_snorkel_event_stats(filtered_sd4m_train)

In [19]:
from wsee.data import pipeline

df_sd_train, Y_sd_train = pipeline.build_event_role_examples(filtered_sd4m_train)

100%|██████████| 7285/7285 [00:02<00:00, 2668.53it/s]


In [20]:
from wsee.labeling import event_argument_role_lfs as role_lfs
from snorkel.labeling import PandasLFApplier

lfs = [
    role_lfs.lf_stanford_separate_sentence,
    role_lfs.lf_somajo_separate_sentence,
    role_lfs.lf_too_far_40,
    role_lfs.lf_somajo_separate_sentence_or_too_far_40
]
applier = PandasLFApplier(lfs)

,j,Polarity,Coverage,Overlaps,Conflicts,Correct,Incorrect,Emp. Acc.
lf_stanford_separate_sentence,0,[10],0.423747,0.315305,0.0,2993,94,0.969550
lf_somajo_separate_sentence,1,[10],0.288813,0.288813,0.0,2097,7,0.996673
lf_too_far_40,2,[10],0.169938,0.169938,0.0,1238,0,1.000000
lf_somajo_separate_sentence_or_too_far_40,3,[10],0.322443,0.322443,0.0,2342,7,0.997020


In [ ]:
L_sd_train = applier.apply(df_sd_train)

In [ ]:
from snorkel.labeling import LFAnalysis

LFAnalysis(L_sd_train, lfs).lf_summary(Y_sd_train)

The SD4M train set contains 2001 positive event roles and 5284 negative event roles.
`lf_stanford_separate_sentence` using the sentence splitting information from stanfordnlp correctly labels 2993 of the negative event roles, but incorrectly labels 94 of the positive event roles as `no_arg`. 
While `lf_somajo_separate_sentence` correctly labels less of of the negative event roles (2097), it only labels 7 of the positive event roles incorrectly. 

Event extraction on a sentence level, i.e. where the input to the model is a sentence not a document, hinges on the quality of the sentence splitting. We do not want to loose these 94 examples of positive event roles for model training.
That is why we decided to feed documents into the model.
As the information from the sentence splitting is crucial to the role labeling functions that only take proximity into account, we chose `lf_somajo_separate_sentence`.

To make sure that those errors are not due to inconsistencies in the annotation, we will manually examine the incorrect instances.

In [23]:
from wsee.preprocessors import preprocessors

def get_simple_trigger(doc):
    trigger = doc.trigger
    simple_trigger = {
        'text': trigger['text'],
        'entity_type': trigger['entity_type'],
        'char_start': trigger['char_start'],
        'char_end': trigger['char_end']
    }
    return simple_trigger

def get_simple_argument(doc):
    argument = doc['argument']
    simple_argument = {
        'text': argument['text'],
        'entity_type': argument['entity_type'],
        'char_start': argument['char_start'],
        'char_end': argument['char_end']
    }
    return simple_argument

def get_simple_somajo_sentences(doc):
    sentences = doc.somajo_doc['sentences']
    return [{'text': doc.text[sentence['char_start']:sentence['char_end']], 'char_start': sentence['char_start'], 'char_end': sentence['char_end']} for sentence in sentences]

def get_simple_stanford_sentences(doc):
    sentences = doc.sentence_spans
    return [{'text': doc.text[sentence['char_start']:sentence['char_end']], 'char_start': sentence['char_start'], 'char_end': sentence['char_end']} for sentence in sentences]

def add_simple_columns(df):
    df['trigger_sm'] = df.apply(lambda doc: get_simple_trigger(doc), axis=1)
    df['argument_sm'] = df.apply(lambda doc: get_simple_argument(doc), axis=1)
    df['somajo_sm'] = df.apply(lambda doc: get_simple_somajo_sentences(doc), axis=1)
    df['stanford_sm'] = df.apply(lambda doc: get_simple_stanford_sentences(doc), axis=1)
    df['between_tokens'] = df.apply(lambda doc: preprocessors.get_between_tokens(doc), axis=1)
    return df

,text,between_tokens,trigger_sm,argument_sm,stanford_sm,event_role
888,"folgende Meldung ergänzt\nam Mittwoch, 6. und Donnerstag, 7. April, jeweils 20.45 – 24.00 Uhr\nMeldung:\nCNL 471 nach Zürich HB (planmäßig 21.33 Uhr ab Berlin Gesundbrunnen) fährt bis zu 46 Min. früher von Berlin-Gesundbrunnen bis Bitterfeld und hält nicht in Halle (Saale) Hbf.\nGrund:\nSoftwareanpassungen im Elektronischen Stellwerk Halle (Saale)\nLink zur detaillierten Meldung: \nLink zum kompletten PDF-Dokument: \n(142 kB)\n------------------\n","[nach, Zürich, HB, (, planmäßig, 21.33, Uhr, ab, Berlin, Gesundbrunnen, ), fährt, bis, zu, 46, Min, ., früher, von, Berlin, -, Gesundbrunnen, bis, Bitterfeld, und]","{'text': 'hält nicht', 'entity_type': 'trigger', 'char_start': 243, 'char_end': 253}","{'text': 'CNL 471', 'entity_type': 'location_route', 'char_start': 102, 'char_end': 109}","[{'text': 'folgende Meldung ergänzt am Mittwoch, 6.', 'char_start': 0, 'char_end': 40}, {'text': 'und Donnerstag, 7.', 'char_start': 41, 'char_end': 59}, {'text': 'April, jeweils 20.45 – 24.00 Uhr Meldung: CNL 471 nach Zürich HB (planmäßig 21.33 Uhr ab Berlin Gesundbrunnen) fährt bis zu 46 Min.', 'char_start': 60, 'char_end': 191}, {'text': 'früher von Berlin-Gesundbrunnen bis Bitterfeld und hält nicht in Halle (Saale) Hbf.', 'char_start': 192, 'char_end': 275}, {'text': 'Grund: Softwareanpassungen im Elektronischen Stellwerk Halle (Saale) Link zur detaillierten Meldung: Link zum kompletten PDF-Dokument: (142 kB) ------------------', 'char_start': 276, 'char_end': 440}]",route
889,"folgende Meldung ergänzt\nam Mittwoch, 6. und Donnerstag, 7. April, jeweils 20.45 – 24.00 Uhr\nMeldung:\nCNL 471 nach Zürich HB (planmäßig 21.33 Uhr ab Berlin Gesundbrunnen) fährt bis zu 46 Min. früher von Berlin-Gesundbrunnen bis Bitterfeld und hält nicht in Halle (Saale) Hbf.\nGrund:\nSoftwareanpassungen im Elektronischen Stellwerk Halle (Saale)\nLink zur detaillierten Meldung: \nLink zum kompletten PDF-Dokument: \n(142 kB)\n------------------\n","[(, planmäßig, 21.33, Uhr, ab, Berlin, Gesundbrunnen, ), fährt, bis, zu, 46, Min, ., früher, von, Berlin, -, Gesundbrunnen, bis, Bitterfeld, und]","{'text': 'hält nicht', 'entity_type': 'trigger', 'char_start': 243, 'char_end': 253}","{'text': 'Zürich HB', 'entity_type': 'location_stop', 'char_start': 115, 'char_end': 124}","[{'text': 'folgende Meldung ergänzt am Mittwoch, 6.', 'char_start': 0, 'char_end': 40}, {'text': 'und Donnerstag, 7.', 'char_start': 41, 'char_end': 59}, {'text': 'April, jeweils 20.45 – 24.00 Uhr Meldung: CNL 471 nach Zürich HB (planmäßig 21.33 Uhr ab Berlin Gesundbrunnen) fährt bis zu 46 Min.', 'char_start': 60, 'char_end': 191}, {'text': 'früher von Berlin-Gesundbrunnen bis Bitterfeld und hält nicht in Halle (Saale) Hbf.', 'char_start': 192, 'char_end': 275}, {'text': 'Grund: Softwareanpassungen im Elektronischen Stellwerk Halle (Saale) Link zur detaillierten Meldung: Link zum kompletten PDF-Dokument: (142 kB) ------------------', 'char_start': 276, 'char_end': 440}]",direction


In [ ]:
from wsee.labeling import error_analysis
from wsee import ROLE_LABELS
pd.set_option('display.max_colwidth', -1)
labeled_sd4m_roles = df_sd_train.copy()
labeled_sd4m_roles['label'] = Y_sd_train
labeled_sd4m_roles['event_role'] = [ROLE_LABELS[label_idx] for label_idx in Y_sd_train]
labeled_sd4m_roles = add_simple_columns(labeled_sd4m_roles)

error_analysis.get_false_positives(labeled_df=labeled_sd4m_roles, lf_outputs=L_sd_train, lf_index=1, label_of_interest=10)[['text', 'between_tokens', 'trigger_sm', 'argument_sm', 'somajo_sm', 'event_role']]

In [25]:
somajo_uncaughts = error_analysis.get_abstained_instances(labeled_df=labeled_sd4m_roles, lf_outputs=L_sd_train, lf_index=1, label_of_interest=10)
somajo_uncaughts_too_far = somajo_uncaughts[somajo_uncaughts['between_distance'] > 40]

In [26]:
somajo_uncaughts_too_far.loc[[3402, 6503]][['text', 'between_tokens', 'trigger_sm', 'argument_sm', 'somajo_sm', 'event_role']]

,text,between_tokens,trigger_sm,argument_sm,somajo_sm,event_role
3402,"veränderte Fahrzeiten Würzburg Hbf / Nürnberg Hbf \n Umleitung und kein Halt in Nürnberg Hbf und Ingolstadt Hbf (12. und 13.03.)\nvon Samstag, 5. März, 23.00 Uhr bis Dienstag, 29. März, 4.00 Uhr\nMeldung:\n Die ICE-Züge verspäten sich von Nürnberg Hbf bis München Hbf um bis zu 10 Min.\n Die ICE-Züge fahren bis zu 10 Min. früher von München Hbf bis Nürnberg Hbf.\n Die ICE-Züge werden am 12. und 13.03. zwischen Würzburg Hbf und München Hbf über Augsburg Hbf umgeleitet und halten nicht in Nürnberg Hbf und Ingolstadt Hbf. Beachten Sie die 20 Min. spätere Ankunft / frühere Abfahrt in München Hbf.\nGrund:\nInbetriebnahme des Elektronischen Stellwerks Fischbach\nLink zur detaillierten Meldung: \nLink zum kompletten PDF-Dokument: \n(191 kB)\nSonderinformation zu dieser Meldung:\nPlakat: Bauarbeiten Nürnberg – Ingolstadt, 06.03. – 28.03.2016 (157 kB)\n------------------\n","[sich, von, Nürnberg, Hbf, bis, München, Hbf, um, bis, zu, 10, Min, ., Die, ICE, -, Züge, fahren, bis, zu, 10, Min, ., früher, von, München, Hbf, bis, Nürnberg, Hbf, ., Die, ICE, -, Züge, werden, am, 12, ., und, 13.03, ., zwischen, Würzburg, Hbf, und]","{'text': 'verspäten', 'entity_type': 'trigger', 'char_start': 216, 'char_end': 225}","{'text': 'München Hbf', 'entity_type': 'location_stop', 'char_start': 424, 'char_end': 435}","[{'text': 'veränderte Fahrzeiten Würzburg Hbf / Nürnberg Hbf Umleitung und kein Halt in Nürnberg Hbf und Ingolstadt Hbf (12. und 13.03.) von Samstag, 5. März, 23.00 Uhr bis Dienstag, 29. März, 4.00 Uhr Meldung: Die ICE-Züge verspäten sich von Nürnberg Hbf bis München Hbf um bis zu 10 Min. Die ICE-Züge fahren bis zu 10 Min. früher von München Hbf bis Nürnberg Hbf. Die ICE-Züge werden am 12. und 13.03. zwischen Würzburg Hbf und München Hbf über Augsburg Hbf umgeleitet und halten nicht in Nürnberg Hbf und Ingolstadt Hbf. Beachten Sie die 20 Min. spätere Ankunft / frühere Abfahrt in München Hbf. Grund: Inbetriebnahme des Elektronischen Stellwerks Fischbach Link zur detaillierten Meldung: Link zum kompletten PDF-Dokument: (191 kB) Sonderinformation zu dieser Meldung: Plakat: Bauarbeiten Nürnberg – Ingolstadt, 06.03. – 28.03.2016 (157 kB) ------------------ ', 'char_start': 0, 'char_end': 860}]",no_arg
6503,"Umleitung und kein Halt in Langenhagen Mitte und Hannover Hbf sowie zusätzlicher Halt in Hannover Messe / Laatzen und Verspätungen \n Verspätungen Celle < > Hannover Hbf\n Zugausfall Hamburg-Altona > Hannover Hbf (09.05.)\nfolgende Meldung entfällt\nvon Sonntag, 1. Mai, 16.00 Uhr bis Mittwoch, 18. Mai, 5.00 Uhr\nMeldung:\n Einige IC-Züge werden umgeleitet und halten nicht in Langenhagen Mitte und Hannover Hbf. Die Züge halten zusätzlich in Hannover Messe / Laatzen und verspäten sich um bis zu 15 Min.\n Einige IC-Züge verspäten sich zwischen Celle und Hannover Hbf sowie darüber hinaus um 10 – 25 Min.\n IC 2179 nach Wolfsburg Hbf (planmäßig 5.05 Uhr ab Hamburg-Altona) fällt am 09.05. von Hamburg-Altona bis Hannover Hbf aus.\nDie Bauarbeiten wurden abgesagt – die Züge verkehren planmäßig.\nLink zur detaillierten Meldung: \nLink zum kompletten PDF-Dokument: \n(196 kB)\n---\n","[sowie, zusätzlicher, Halt, in, Hannover, Messe, /, Laatzen, und, Verspätungen, Verspätungen, Celle, <, >, Hannover, Hbf, Zugausfall, Hamburg, -, Altona, >, Hannover, Hbf, (, 09.05, ., ), folgende, Meldung, entfällt, von, Sonntag, ,, 1, ., Mai, ,, 16.00, Uhr, bis, Mittwoch, ,, 18, ., Mai, ,, 5.00, Uhr, Meldung, :, Einige, IC, -, Züge, werden, umgeleitet, und, halten, nicht, in, Langenhagen, Mitte, und, Hannover, Hbf, ., Die, Züge, halten, zusätzlich, in, Hannover, Messe, /, Laatzen, und, verspäten, sich, um, bis, zu, 15, Min, ., Einige, IC, -, Züge]","{'text': 'verspäten', 'entity_type': 'trigger', 'char_start': 516, 'char_end': 525}","{'text': 'Hannover Hbf', 'entity_type': 'location_stop', 'char_start': 49, 'char_end': 61}","[{'text': 'Umleitung und kein Halt in Langenhagen Mitte und Hannover Hbf sowie